In [13]:
"""
CommandCenter File
read from influx and publish to mqtt broker
"""

# Importing relevant modules
import os
from dotenv import load_dotenv
from influxdb_client import InfluxDBClient, Point, Dialect
from influxdb_client.client.write_api import ASYNCHRONOUS
import paho.mqtt.client as mqtt
import time
import json
import requests
import pandas as pd

# Load environment variables from ".env"
load_dotenv()

# InfluxDB config
BUCKET = 'fullstack-influxdb' #  bucket is a named location where time series data is stored.
url = 'https://iot-group2-service1.iotcloudserve.net/'
token ='Dwj0HPIYScc1zvkB0zHpjxIVIssU_z_-unniio7sOcZl135FZ40ONj9ZX6jgiBWqkwpOQegRAL21Ix1z86SBJw=='
org = 'Chulalongkorn'
print("connecting to",url)

client = InfluxDBClient(

    url= url,
    token= token,
    org= 'Chulalongkorn'
)
write_api = client.write_api()
query_api = client.query_api()

#"temp_sht4x": 34.64,"humid_sht4x": 36.80,"temp_bmp280": 35.90,"pressure_bmp280": 1006.34}


query2 = 'from(bucket: "fullstack-influxdb")\
    |> range(start: -2d) \
    |> filter(fn: (r) => r._measurement == "sensor_data")\
    |> filter(fn: (r) => r._field == "humid_sht4x" or r._field == "pressure_bmp280" or r._field == "temp_bmp280" or r._field == "temp_sht4x")\
    |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")'


csv_result = query_api.query_csv(query2,
                                 dialect=Dialect(header=False, delimiter=",", comment_prefix="#", annotations=[],
                                                 date_time_format="RFC3339"))


#csv_result =  query_api.query_csv(query2)
df1 = pd.DataFrame(csv_result)
print("Successfully convert to DataFrame")
#print(df1.iloc[:-5])
#df.to_csv('RaspberryPi/CommandCenter/12_5_12_50.csv', index=False)

client.close()


connecting to https://iot-group2-service1.iotcloudserve.net/
Successfully convert to DataFrame


In [3]:
# display all column
pd.set_option('display.max_columns', None)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10
0,,_result,0,2024-05-08T04:24:49.644466628Z,2024-05-13T04:24:49.644466628Z,2024-05-12T04:47:38.47860882Z,sensor_data,56.32,1009.07,38.51,38.45
1,,_result,0,2024-05-08T04:24:49.644466628Z,2024-05-13T04:24:49.644466628Z,2024-05-12T04:47:40.159709834Z,sensor_data,47.2,1009,42.34,42.66
2,,_result,0,2024-05-08T04:24:49.644466628Z,2024-05-13T04:24:49.644466628Z,2024-05-12T04:47:42.154275027Z,sensor_data,46.5,1008.92,42.54,42.95
3,,_result,0,2024-05-08T04:24:49.644466628Z,2024-05-13T04:24:49.644466628Z,2024-05-12T04:47:44.154988015Z,sensor_data,46.52,1008.84,42.52,42.82
4,,_result,0,2024-05-08T04:24:49.644466628Z,2024-05-13T04:24:49.644466628Z,2024-05-12T04:47:46.180702931Z,sensor_data,46.48,1008.7,42.52,42.82


In [10]:
# drop column
columns_to_keep = [5, 7, 8, 9, 10]
df2 = df1[df1.columns[columns_to_keep]]
df2

,5,7,8,9,10
0,2024-05-12T04:47:38.47860882Z,56.32,1009.07,38.51,38.45
1,2024-05-12T04:47:40.159709834Z,47.2,1009,42.34,42.66
2,2024-05-12T04:47:42.154275027Z,46.5,1008.92,42.54,42.95
3,2024-05-12T04:47:44.154988015Z,46.52,1008.84,42.52,42.82
4,2024-05-12T04:47:46.180702931Z,46.48,1008.7,42.52,42.82
...,...,...,...,...,...
3831,2024-05-13T04:09:37.204720622Z,40.8,1009.02,33.74,32.97
3832,2024-05-13T04:10:10.219543194Z,41,1009.05,33.75,32.82
3833,2024-05-13T04:10:10.31675782Z,41.15,1009.03,33.72,32.84
3834,2024-05-13T04:10:43.602726001Z,41.15,1009.03,33.72,32.84


In [11]:
new_column_names = ['time', 'humid_sht4x', 'pressure_bmp280', 'temp_bmp280', 'temp_sht4x']

# Rename the columns
df3 = df2.rename(columns=dict(zip(df2.columns, new_column_names)), inplace=False)
df3

,time,humid_sht4x,pressure_bmp280,temp_bmp280,temp_sht4x
0,2024-05-12T04:47:38.47860882Z,56.32,1009.07,38.51,38.45
1,2024-05-12T04:47:40.159709834Z,47.2,1009,42.34,42.66
2,2024-05-12T04:47:42.154275027Z,46.5,1008.92,42.54,42.95
3,2024-05-12T04:47:44.154988015Z,46.52,1008.84,42.52,42.82
4,2024-05-12T04:47:46.180702931Z,46.48,1008.7,42.52,42.82
...,...,...,...,...,...
3831,2024-05-13T04:09:37.204720622Z,40.8,1009.02,33.74,32.97
3832,2024-05-13T04:10:10.219543194Z,41,1009.05,33.75,32.82
3833,2024-05-13T04:10:10.31675782Z,41.15,1009.03,33.72,32.84
3834,2024-05-13T04:10:43.602726001Z,41.15,1009.03,33.72,32.84


In [12]:
# Convert 'time' column to datetime format
df3['time'] = pd.to_datetime(df3['time'])

# Format 'time' column as desired
df3['time'] = df3['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
df3

,time,humid_sht4x,pressure_bmp280,temp_bmp280,temp_sht4x
0,2024/05/12 04:47:38,56.32,1009.07,38.51,38.45
1,2024/05/12 04:47:40,47.2,1009,42.34,42.66
2,2024/05/12 04:47:42,46.5,1008.92,42.54,42.95
3,2024/05/12 04:47:44,46.52,1008.84,42.52,42.82
4,2024/05/12 04:47:46,46.48,1008.7,42.52,42.82
...,...,...,...,...,...
3831,2024/05/13 04:09:37,40.8,1009.02,33.74,32.97
3832,2024/05/13 04:10:10,41,1009.05,33.75,32.82
3833,2024/05/13 04:10:10,41.15,1009.03,33.72,32.84
3834,2024/05/13 04:10:43,41.15,1009.03,33.72,32.84
